# Import Packages

In [22]:
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import nba_api.stats.endpoints as nba
import time

# Connect to MySQL Database

In [23]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

# Query game info to call NBA API for box scores

In [3]:
select_statement = """SELECT GAME_ID, GAME_DATE FROM nba.game_logs WHERE SEASON_ID = '22018';"""
cursor.execute(select_statement)
game_ids = cursor.fetchall()

In [4]:
len(game_ids)

2460

In [5]:
# Get the set of all the game_ids
game_ids = set(game_ids)
game_ids_df = pd.DataFrame(game_ids)
game_ids_df.columns = ['GAME_ID', 'DATE']

# Sort by GAME_ID and reset index
game_ids_df.sort_values(by = 'GAME_ID', inplace = True)
game_ids_df.reset_index(drop = True, inplace = True)

In [7]:
game_ids_df.head()

,GAME_ID,DATE
0,0021800001,2018-10-16
1,0021800002,2018-10-16
2,0021800003,2018-10-17
3,0021800004,2018-10-17
4,0021800005,2018-10-17


In [8]:
def get_box_score(game_id):
    try:
        box_score = nba.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
        box_score['pk'] = str(box_score['PLAYER_ID']) + box_score['GAME_ID']
        print(box_score.head())
        return box_score
    except:
        return 'Error'

In [9]:
player_stats = []
for game in game_ids_df['GAME_ID']:
    player_stats.append(get_box_score(game))

      GAME_ID     TEAM_ID TEAM_ABBREVIATION     TEAM_CITY  PLAYER_ID  \
0  0021800001  1610612755               PHI  Philadelphia     203967   
1  0021800001  1610612755               PHI  Philadelphia     203496   
2  0021800001  1610612755               PHI  Philadelphia     203954   
3  0021800001  1610612755               PHI  Philadelphia    1628365   
4  0021800001  1610612755               PHI  Philadelphia    1627732   

        PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB   REB  AST  \
0       Dario Saric              F          22:54  3.0  ...   6.0   6.0  1.0   
1  Robert Covington              F          34:13  3.0  ...   5.0   6.0  0.0   
2       Joel Embiid              C          36:49  9.0  ...   8.0  10.0  2.0   
3    Markelle Fultz              G          24:20  2.0  ...   3.0   3.0  2.0   
4       Ben Simmons              G          42:44  7.0  ...  12.0  15.0  8.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  0.0  0.0  3.0  5.0   6.0        -4.0   


      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021800007  1610612737               ATL   Atlanta       1713   
1  0021800007  1610612737               ATL   Atlanta    1627752   
2  0021800007  1610612737               ATL   Atlanta     203458   
3  0021800007  1610612737               ATL   Atlanta     203145   
4  0021800007  1610612737               ATL   Atlanta    1629027   

      PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB  REB  AST  \
0    Vince Carter              F          23:58  4.0  ...   2.0  2.0  0.0   
1  Taurean Prince              F          32:19  7.0  ...   6.0  6.0  6.0   
2        Alex Len              C          21:27  3.0  ...   2.0  4.0  2.0   
3   Kent Bazemore              G          30:16  6.0  ...   4.0  6.0  0.0   
4      Trae Young              G          33:08  5.0  ...   6.0  6.0  5.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  0.0  0.0  2.0  5.0  12.0         7.0   
1  0.0  0.0  6.0  4.0  21.0       -23.0   

      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021800013  1610612742               DAL    Dallas    1627827   
1  0021800013  1610612742               DAL    Dallas    1629029   
2  0021800013  1610612742               DAL    Dallas     201599   
3  0021800013  1610612742               DAL    Dallas     202083   
4  0021800013  1610612742               DAL    Dallas    1628372   

           PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB   REB  \
0  Dorian Finney-Smith              F          30:20  4.0  ...   3.0   3.0   
1          Luka Doncic              F          31:47  5.0  ...   6.0   8.0   
2       DeAndre Jordan              C          30:30  5.0  ...   8.0  12.0   
3      Wesley Matthews              G          34:21  4.0  ...   1.0   2.0   
4     Dennis Smith Jr.              G          27:54  6.0  ...   1.0   2.0   

   AST  STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  1.0  0.0  3.0  0.0  2.0  12.0       -16.0   
1  4.0  0.0  0.0  4.0  3.0

      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021800019  1610612738               BOS    Boston    1628369   
1  0021800019  1610612738               BOS    Boston     202330   
2  0021800019  1610612738               BOS    Boston     201143   
3  0021800019  1610612738               BOS    Boston    1627759   
4  0021800019  1610612738               BOS    Boston     202681   

      PLAYER_NAME START_POSITION COMMENT    MIN   FGM  ...  DREB   REB  AST  \
0    Jayson Tatum              F          36:47   6.0  ...   9.0   9.0  2.0   
1  Gordon Hayward              F          24:17   6.0  ...   5.0   7.0  2.0   
2      Al Horford              C          33:25   5.0  ...   5.0  10.0  9.0   
3    Jaylen Brown              G          24:10   5.0  ...   3.0   4.0  0.0   
4    Kyrie Irving              G          35:56  10.0  ...   2.0   4.0  6.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  1.0  1.0  5.0  5.0  16.0       -14.0   
1  1.0  0.0  1.0  0.0  14.0   

      GAME_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_CITY  PLAYER_ID  \
0  0021800025  1610612760               OKC  Oklahoma City     202331   
1  0021800025  1610612760               OKC  Oklahoma City     202335   
2  0021800025  1610612760               OKC  Oklahoma City     203500   
3  0021800025  1610612760               OKC  Oklahoma City    1628390   
4  0021800025  1610612760               OKC  Oklahoma City     203471   

         PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB   REB  AST  \
0        Paul George              F          38:07  7.0  ...   7.0   8.0  4.0   
1  Patrick Patterson              F          19:27  2.0  ...   1.0   2.0  0.0   
2       Steven Adams              C          33:18  8.0  ...  12.0  18.0  0.0   
3  Terrance Ferguson              G          17:15  1.0  ...   0.0   0.0  0.0   
4    Dennis Schroder              G          30:01  2.0  ...   2.0   2.0  8.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  2.0  0.0  3.0  6.0  20.0    

...

      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021801201  1610612742               DAL    Dallas    1628382   
1  0021801201  1610612742               DAL    Dallas       1717   
2  0021801201  1610612742               DAL    Dallas     203939   
3  0021801201  1610612742               DAL    Dallas     201584   
4  0021801201  1610612742               DAL    Dallas    1628973   

      PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB  REB   AST  \
0  Justin Jackson              F          18:59  6.0  ...   0.0  1.0   1.0   
1   Dirk Nowitzki              F          11:52  1.0  ...   5.0  5.0   1.0   
2   Dwight Powell              C          21:09  7.0  ...   3.0  4.0   1.0   
3    Courtney Lee              G          17:13  0.0  ...   2.0  2.0   1.0   
4   Jalen Brunson              G          23:40  4.0  ...   2.0  2.0  10.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  0.0  0.0  0.0  1.0  16.0        -5.0   
1  0.0  1.0  0.0  0.0   2.0         

      GAME_ID     TEAM_ID TEAM_ABBREVIATION    TEAM_CITY  PLAYER_ID  \
0  0021801207  1610612740               NOP  New Orleans    1629026   
1  0021801207  1610612740               NOP  New Orleans    1626174   
2  0021801207  1610612740               NOP  New Orleans    1626143   
3  0021801207  1610612740               NOP  New Orleans     203546   
4  0021801207  1610612740               NOP  New Orleans     203901   

        PLAYER_NAME START_POSITION COMMENT    MIN   FGM  ...  DREB   REB  \
0  Kenrich Williams              F          28:33   5.0  ...   5.0   5.0   
1    Christian Wood              F          23:47   5.0  ...   4.0   7.0   
2     Jahlil Okafor              C          35:14  10.0  ...   8.0  14.0   
3         Ian Clark              G          35:31  11.0  ...   2.0   4.0   
4     Elfrid Payton              G          39:18  10.0  ...   6.0   6.0   

    AST  STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0   5.0  0.0  2.0  3.0  3.0  12.0        16.0   
1   0.0  0.0  1.0 

      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021801213  1610612752               NYK  New York    1628995   
1  0021801213  1610612752               NYK  New York    1628436   
2  0021801213  1610612752               NYK  New York    1629011   
3  0021801213  1610612752               NYK  New York    1628422   
4  0021801213  1610612752               NYK  New York    1628372   

         PLAYER_NAME START_POSITION COMMENT    MIN   FGM  ...  DREB   REB  \
0      Kevin Knox II              F          34:25   5.0  ...   8.0  10.0   
1        Luke Kornet              F          38:46   5.0  ...   9.0  13.0   
2  Mitchell Robinson              C          35:01   3.0  ...  14.0  17.0   
3     Damyean Dotson              G          40:59   5.0  ...   5.0   5.0   
4   Dennis Smith Jr.              G          34:57  10.0  ...   1.0   1.0   

   AST  STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  2.0  0.0  0.0  1.0  2.0  17.0        12.0   
1  4.0  0.0  6.0  0.0  2.0  12.0

      GAME_ID     TEAM_ID TEAM_ABBREVIATION TEAM_CITY  PLAYER_ID  \
0  0021801219  1610612757               POR  Portland     203090   
1  0021801219  1610612757               POR  Portland     202329   
2  0021801219  1610612757               POR  Portland     202683   
3  0021801219  1610612757               POR  Portland     203468   
4  0021801219  1610612757               POR  Portland     203081   

        PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB   REB  AST  \
0  Maurice Harkless              F          33:59  9.0  ...   5.0   8.0  0.0   
1   Al-Farouq Aminu              F          19:24  0.0  ...   5.0   5.0  0.0   
2       Enes Kanter              C          36:40  8.0  ...  10.0  16.0  2.0   
3       CJ McCollum              G          29:23  5.0  ...   1.0   1.0  3.0   
4    Damian Lillard              G          35:57  6.0  ...   3.0   4.0  8.0   

   STL  BLK   TO   PF   PTS  PLUS_MINUS  \
0  0.0  4.0  0.0  2.0  26.0        13.0   
1  1.0  0.0  0.0  0.0   

      GAME_ID     TEAM_ID TEAM_ABBREVIATION     TEAM_CITY  PLAYER_ID  \
0  0021801225  1610612744               GSW  Golden State    1628395   
1  0021801225  1610612744               GSW  Golden State     201142   
2  0021801225  1610612744               GSW  Golden State     101106   
3  0021801225  1610612744               GSW  Golden State     202691   
4  0021801225  1610612744               GSW  Golden State    1626188   

     PLAYER_NAME START_POSITION COMMENT    MIN  FGM  ...  DREB  REB  AST  STL  \
0    Jordan Bell              F          27:45  6.0  ...   6.0  8.0  1.0  0.0   
1   Kevin Durant              F          18:28  7.0  ...   0.0  0.0  6.0  1.0   
2   Andrew Bogut              C          17:46  0.0  ...   5.0  6.0  1.0  1.0   
3  Klay Thompson              G          16:52  8.0  ...   1.0  1.0  3.0  1.0   
4     Quinn Cook              G          27:47  4.0  ...   4.0  4.0  3.0  0.0   

   BLK   TO   PF   PTS  PLUS_MINUS  \
0  1.0  2.0  2.0  15.0        -9.0   
1  0

In [12]:
# Concatenate box scores into dataframe, reset index
all_stats_2018 = pd.concat(player_stats)
all_stats_2018.reset_index(drop = True, inplace = True)

# Insert box scores into MySQL

In [17]:
# Get nba.box_score columns from MySQL
cursor.execute('''SELECT * FROM nba.box_score''')
box_score = cursor.fetchall()
box_score_df = pd.DataFrame(box_score)

# Get column names for flights dataframe
field_names = [i[0] for i in cursor.description]

In [19]:
import numpy as np
# Reorder dataframe according to order of mmySQL table
cols_sql = field_names[:-1]
cols_sql.append('TO')
game_stats_sql = all_stats_2018[cols_sql]

# Rework primary key variable that was not coded properly earlier
game_stats_sql['pk'] = [x[0] + str(x[4]) for x in game_stats_sql.values]

# Replace nan values with None
game_stats_sql.replace({np.nan: None}, inplace = True)

# Convert each row into a tuple
stats = [tuple(row) for row in game_stats_sql.values]

In [20]:
# Create string of column names and %s necessary for mySQL query
cols_insert = ''
str_insert = ''
for col in field_names:
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

In [21]:
# Insert and commit all the player stats
insert_statement = """INSERT INTO nba.box_score ({}) VALUES ({})""".format(cols_insert, str_insert)
cursor.executemany(insert_statement, stats)
cnx.commit()

In [24]:
select_statement = """SELECT GAME_ID FROM nba.box_score;"""
cursor.execute(select_statement)
stats_ids = cursor.fetchall()

In [29]:
len(stats)

30808

In [30]:
cursor.close()
cnx.close()